In [5]:
import pandas as pd
import math as m

In [6]:
# Mise en forme des données

import pandas as pd
import numpy as np


def file_to_dataframe(filenames): 
    """
    Initialize the DataFrame from a filenames 
    Parameters: 
    -----------
    filenames: str, the name of the file
    st: str, helps to delimites the columns of the datas because "Correspondances_promo
    are delimited with ';' while the other are delimited with ','.
    """
    datas = pd.read_csv(filenames)
    return datas


def clean_dates(df):
    """
    Clean the dates of the files with dates such as "2022-11-14T00:00:00Z" and change it into "2022-11-14"
    -----------
    df: dataframe, the dataframe that we use
    
    """
    for colu in df.columns :
        if df[colu].dtype == object: 
            df[colu] = df[colu].str.replace('T00:00:00Z', '')
    return(df)

def change_date(st):
    """
    Change a date as "28/04/2003 00:00:00.000" into one more precise "2003-04-28"
    -----------
    st: str, the date to change

    """ 
    if type(st) != str or len(st) < 10 : 
        return(st)
    else :
        date = st[:10].split("/")
        return(date[2]+"-"+date[1]+"-"+date[0])
    
def change_dates_all(df,cols):
    """
    Change the dates of some columns of a dataFrame using the function change_date
    -----------
    df: dataframe, the DataFrame on which we are working
    cols: str list, a list of columns name that we want to change

    """ 
    for colu in cols : 
        df[colu] = df[colu].apply(change_date)
    return df

def save_to_csv_file(df,filename):
    """
    Save a dataframe on a .csv file at filename
    ----------
    df: dataframe, the DataFrame that you want to save
    filename: str, the place where you want to save your data frame
    """
    df.to_csv(filename, index=False)
    return True

In [7]:
def join_dataFrames(df1,df2,cond):
    """
    Join two DataFrames on the conditions cond
    Parameters: 
    -----------
    df1,df2: DataFrames
    cond: str list of the colomuns on which we want to join 
    """
    return pd.merge(df1, df2, on=cond, how='inner')

def join_dataFrames_outer(df1,df2,cond):
    """
    Join two DataFrames on the conditions cond
    Parameters: 
    -----------
    df1,df2: DataFrames
    cond: str list of the colomuns on which we want to join 
    """
    return pd.merge(df1, df2, on=cond, how='outer')



In [4]:

#1)

def df_filter_condition(df,str_name, str_cond):
    """
    This function filters the elemenent of the columns str_name with the condition ==
    str_cond
    """
    return df[df[str_name] == str_cond]


def df_filter_begin_name(df, str_name, str_filter):
    """
    This function filters the elemenent of the columns str_name which begins with 
    str_filter
    """
    return df[df[str_name].str.startswith(str_filter)]


def apply_conditions(row):
    """
    This function creates the elements of a new column where the promos are put
    together in function of some conditions.
    """
    nb_days = 'NBJOUR_ODD'
    carticle = 'CARTICLE_ODD'
    if row['SG'] == 'Semaine généreuse':
        return 'Semaine genéreuse'
    elif row[nb_days] == 7 :
        return 'ODD 7 jours autre que SG'
    elif (row[carticle][:2] == 'EV') & (row[nb_days] == 15) :
        return 'ODD 15 jours EV+'
    elif (row[carticle][:2]  == 'TC') & (row[nb_days] == 15) :
        return 'ODD 15 jours TC'
    elif (row[carticle][:2] == 'EV') & (row[nb_days] == 21) :
        return 'ODD 21 jours EV+ '
    elif (row[carticle][:2] == 'TC') & (row[nb_days] == 21):
        return 'ODD 21 jours EV+ '
    else: 
        return 'Autres'
        
def create_new_column(df: pd.DataFrame,function):
    """
    This function is used to sendback a new column using the function apply
    and applying function to the elements 
    """
    return df.apply(function,axis = 1)

def keep_used_odd(df_Données_Promos,df_odd,n) :
    """
    This function delete all the promos which are not used enough
    """
    df_join_odd = join_dataFrames(df_Données_Promos,df_odd,['CPROMO'])

    series_count = df_join_odd.groupby(['CPROMO'])['ID_ABONNE'].count() #count the number of people who used each PROMO
    series_count = series_count.sort_values(ascending=False) #sort the number of people
    series_count = series_count[series_count > n ] # Keep the more used
    series_count = series_count.rename('NOMBRE_UTILISATION') # Rename with a proper name

    df_filtered_groups = series_count.reset_index() # series to Data fram
    df_new_odd = join_dataFrames(df_filtered_groups,df_odd,['CPROMO'])

    return df_new_odd

def str_to_date(df,col_name):
    return pd.to_datetime(df[col_name])



In [16]:
#Test

#1)

def creation_df_odd() :
    """
    This function creates the new df_odd on your Computer
    """
    df_Correspondances_Promos = file_to_dataframe("/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/Correspondances_Promos.csv") 

    df_odd = df_filter_condition(df_Correspondances_Promos,'TYPE_PROMO','ODD') #we create a DataFrame with only ODD Promotion
    df_odd['TYPE_PROMON'] = create_new_column(df_odd,apply_conditions) #we create new columns on this DataFrame of the ODD type


    #creation df_odd_2021
    df_Données_Promos_2021 = file_to_dataframe("/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/Données_Promos_2021.csv")

    n = df_Données_Promos_2021.shape[0] / 10000 #number minimum of used
    df_new_odd = keep_used_odd(df_Données_Promos_2021,df_odd,n) #creation of the new tab by keeping only the used promos
    
    save_to_csv_file(df_new_odd,"/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/odd_2021.csv")

    #creation df_odd_2022  
    df_Données_Promos_2022 = file_to_dataframe("/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/Données_Promos_2022.csv")

    n = df_Données_Promos_2022.shape[0] / 10000 #number minimum of used
    df_new_odd = keep_used_odd(df_Données_Promos_2022,df_odd,n) #creation of the new tab by keeping only the used promos
    
    save_to_csv_file(df_new_odd,"//Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/odd_2022.csv")
    
    #creation df_odd_2023
    df_Données_Promos_2023 = file_to_dataframe("/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/Données_Promos_2023.csv")

    n = df_Données_Promos_2023.shape[0] / 10000 #number minimum of used
    df_new_odd = keep_used_odd(df_Données_Promos_2023,df_odd,n) #creation of the new tab by keeping only the used promos
    
    save_to_csv_file(df_new_odd,"/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/odd_2023.csv")

    return True

In [6]:
def str_to_date(df,col_name):
    return pd.to_datetime(df[col_name])

In [12]:
df_Données_Promos_2021 = file_to_dataframe("/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/Données_Promos_2021.csv")
df_odd = file_to_dataframe("/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/odd_2021.csv")
df_odd_new = df_odd[['CPROMO','TYPE_PROMON']] 

df_join = join_dataFrames(df_Données_Promos_2021,df_odd_new,['CPROMO'])
series = df_join.groupby('TYPE_PROMON')['ID_ABONNE'].count()
series

TYPE_PROMON
Autres                       343172
ODD 15 jours EV+              83607
ODD 15 jours TC             1375984
ODD 21 jours EV+              68290
ODD 7 jours autre que SG      39041
Semaine genéreuse           2340995
Name: ID_ABONNE, dtype: int64

In [42]:
df_Données_Promos_2022 = file_to_dataframe("/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/Données_Promos_2022.csv")
df_odd = file_to_dataframe("/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/odd_2022.csv")
df_odd_new = df_odd[['CPROMO','TYPE_PROMON']] 

df_join = join_dataFrames(df_Données_Promos_2022,df_odd_new,['CPROMO'])
series = df_join.groupby('TYPE_PROMON')['ID_ABONNE'].count()
series

TYPE_PROMON
Autres                       785020
ODD 15 jours EV+               6883
ODD 15 jours TC             1796235
ODD 21 jours EV+             210628
ODD 7 jours autre que SG      52213
Semaine genéreuse           3067877
Name: ID_ABONNE, dtype: int64

In [41]:
df_Données_Promos_2023 = file_to_dataframe("/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/Données_Promos_2023.csv")
df_odd = file_to_dataframe("/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/odd_2023.csv")
df_odd_new = df_odd[['CPROMO','TYPE_PROMON']] 

df_join = join_dataFrames(df_Données_Promos_2023,df_odd_new,['CPROMO'])
series = df_join.groupby('TYPE_PROMON')['ID_ABONNE'].count()
series

TYPE_PROMON
Autres                       130807
ODD 15 jours EV+               9736
ODD 15 jours TC             1998454
ODD 21 jours EV+             155165
ODD 7 jours autre que SG      34033
Semaine genéreuse           2543931
Name: ID_ABONNE, dtype: int64

In [8]:
# 2

# Filtrer par type de promo, nouvelle colonne ? 

def df_mois_annee(df) :

    df['DATE_ACTE_REEL'] = pd.to_datetime(df['DATE_ACTE_REEL'])
    
    df['MONTH'] = df['DATE_ACTE_REEL'].dt.month
    df['YEAR'] = df['DATE_ACTE_REEL'].dt.year

    return df


In [38]:

# Exo 2 Test pour 'CANAL_DISTRIB' à faire en +  : Enregistrer sur un csv + run pour les autres conditions 

def repartition_reabo_2021():

    df_Données_Promos_2021 = file_to_dataframe("/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/Données_Promos_2021.csv")
    df_Données_Reabos_2021 = file_to_dataframe("/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/Données_Reabos_2021.csv")
    df_odd = file_to_dataframe("/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/odd_2021.csv")
    df_odd_new = df_odd[['CPROMO','TYPE_PROMON']] 

    df_join = join_dataFrames(df_Données_Promos_2021,df_odd_new,['CPROMO'])
    df_join = join_dataFrames(df_join,df_Données_Reabos_2021,['ID_ABONNE','DATE_ACTE_REEL'])
    df_join = df_mois_annee(df_join)

    series = df_join.groupby(['TYPE_PROMON', 'CANAL_DISTRIB'])['ID_ABONNE'].count()
    df_filtered_groups = series.reset_index(name = 'NB')

    save_to_csv_file(df_filtered_groups,"/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/type_promo_canaldistrib.csv")

    series = df_join.groupby(['MONTH', 'YEAR', 'CANAL_DISTRIB'])['ID_ABONNE'].count()
    df_filtered_groups = series.reset_index(name = 'NB')

    save_to_csv_file(df_filtered_groups,"/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/month_canaldistrib.csv")

    series = df_join.groupby(['TYPE_PROMON','MONTH', 'YEAR', 'CANAL_DISTRIB'])['ID_ABONNE'].count()
    df_filtered_groups = series.reset_index(name = 'NB')

    save_to_csv_file(df_filtered_groups,"/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/repartition_canaldistrib.csv")

    series = df_join.groupby(['TYPE_PROMON','MONTH','YEAR', 'REGION'])['ID_ABONNE'].count()
    df_filtered_groups = series.reset_index(name = 'NB')

    save_to_csv_file(df_filtered_groups,"/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/repartition_region.csv")

    series = df_join.groupby(['TYPE_PROMON','MONTH','YEAR', 'SECTEUR'])['ID_ABONNE'].count()
    df_filtered_groups = series.reset_index(name = 'NB')

    save_to_csv_file(df_filtered_groups,"/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/repartition_secteur.csv")

    series = df_join.groupby(['TYPE_PROMON','MONTH','YEAR', 'ENSEIGNE'])['ID_ABONNE'].count()
    df_filtered_groups = series.reset_index(name = 'NB')

    save_to_csv_file(df_filtered_groups,"/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/repartition_enseigne.csv")

    series = df_join.groupby(['TYPE_PROMON','MONTH','YEAR', 'MOYEN_PAIEMENT'])['ID_ABONNE'].count()
    df_filtered_groups = series.reset_index(name = 'NB')

    save_to_csv_file(df_filtered_groups,"/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/repartition_moypay.csv")

    series = df_join.groupby(['TYPE_PROMON','MONTH','YEAR', 'FORMULE_PREC'])['ID_ABONNE'].count()
    df_filtered_groups = series.reset_index(name = 'NB')

    save_to_csv_file(df_filtered_groups,"/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/repartition_formule.csv")

    
    return True

In [43]:
 """
 df_Données_Promos_2021 = file_to_dataframe("/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/Données_Promos_2021.csv")
 df_Données_Reabos_2021 = file_to_dataframe("/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/Données_Reabos_2021.csv")
 df_odd = file_to_dataframe("/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/odd_2021.csv")
 df_odd_new = df_odd[['CPROMO','TYPE_PROMON']] 

 df_join = join_dataFrames(df_Données_Promos_2021,df_odd_new,['CPROMO'])
 df_join = join_dataFrames(df_join,df_Données_Reabos_2021,['ID_ABONNE','DATE_ACTE_REEL'])
 df_join = df_mois_annee(df_join)

 series = df_join.groupby(['TYPE_PROMON', 'MONTH'])['ID_ABONNE'].count()
 df_filtered_groups = series.reset_index(name = 'NB')
 df_filtered_groups

"""


,TYPE_PROMON,MONTH,NB
0,Autres,1,38216
1,Autres,2,19186
2,Autres,3,14696
3,Autres,4,4153
4,Autres,5,1454
5,Autres,6,60431
6,Autres,7,25090
7,Autres,8,5833
8,Autres,9,3378
9,Autres,10,2454


In [39]:
repartition_reabo_2021()

True

In [ ]:

"""
print(df_join.groupby(['TYPE_PROMON','CANAL_DISTRIB'])['ID_ABONNE'].count())
print(df_join.groupby(['MONTH','CANAL_DISTRIB'])['ID_ABONNE'].count())
print(df_join.groupby(['TYPE_PROMON','MONTH','CANAL_DISTRIB'])['ID_ABONNE'].count())
"""   


In [19]:
#3)

def str_to_date(df,col_name):
    return pd.to_datetime(df[col_name])


def time_reabo_columns(df,end_abo,date_reabo):
    """
    This function creates a column which represents the delay between the end 
    of the last abo and  the act of reabo
    """
    df[date_reabo] = str_to_date(df,date_reabo)
    df[end_abo] = str_to_date(df,end_abo)
    return (df[date_reabo] - df[end_abo]).dt.days

def count_abo_conditions(df,name_col,cond):
    """
    This function count the number of elements of the condition and group by
    the column 
    """
    return df.groupby(name_col)[cond].count()

def mean_time_reabo(df,name_col,cond):
    """
    This function calculate the mean of the condition and group by
    the column 
    """
    return df.groupby([name_col])[cond].mean()

def mean_empty_col(df,col,cond):
    """
    This function calculate the mean of the datas of column col by dividing
    between the datas where the columns cond is empty or not    
    """
    groupes = df.groupby(df[col].isna())
    return groupes[cond].mean()



In [24]:

# Application 

def repartition_reabo_type():

    df_Données_Promos_2021 = file_to_dataframe("/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/Données_Promos_2021.csv")
    df_Données_Reabos_2021 = file_to_dataframe("/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/Données_Reabos_2021.csv")
    df_odd = file_to_dataframe("/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/odd_2021.csv")
    df_odd_new = df_odd[['CPROMO','TYPE_PROMON']] 

    df_Données_Promos_2021 = join_dataFrames(df_odd_new,df_Données_Promos_2021,['CPROMO'])


    end_abo = 'DATE_FIN_ABO_PREC'
    date_reabo = 'DATE_ACTE_REEL'

    df_Données_Reabos_2021['TIME_REABO'] = time_reabo_columns(df_Données_Reabos_2021,end_abo,date_reabo) #Creation new colum "TIME_REABO"

    df_Données_Promos_2021['DATE_ACTE_REEL'] = str_to_date(df_Données_Promos_2021,'DATE_ACTE_REEL') #Preparation join df_Données_Promos_2021 et df_Données_Reabos_2021
    df_join = join_dataFrames(df_Données_Promos_2021,df_Données_Reabos_2021,['ID_ABONNE','DATE_ACTE_REEL']) #Join df_Données_Promos_2021 et df_Données_Reabos_2021

    series_count = mean_time_reabo(df_join,'TYPE_PROMON','TIME_REABO') #Creation of a Serie with the mean of the Reabo by CPROMO
    df_filtered_groups = series_count.reset_index() #Changing Series into a  DataFrame 

    df_filtered_groups = df_filtered_groups.sort_values(by = 'TIME_REABO', ascending=False)

    save_to_csv_file(df_filtered_groups,"/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/mean_time_reabo_promos.csv") # Save on My Mac

    return True


In [25]:

# Application 

def repartition_reabo_code():

    df_Données_Promos_2021 = file_to_dataframe("/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/Données_Promos_2021.csv")
    df_Données_Reabos_2021 = file_to_dataframe("/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/Données_Reabos_2021.csv")
    df_odd = file_to_dataframe("/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/odd_2021.csv")
    df_odd_new = df_odd[['CPROMO','TYPE_PROMON']] 

    df_Données_Promos_2021 = join_dataFrames(df_odd_new,df_Données_Promos_2021,['CPROMO'])


    end_abo = 'DATE_FIN_ABO_PREC'
    date_reabo = 'DATE_ACTE_REEL'

    df_Données_Reabos_2021['TIME_REABO'] = time_reabo_columns(df_Données_Reabos_2021,end_abo,date_reabo) #Creation new colum "TIME_REABO"

    df_Données_Promos_2021['DATE_ACTE_REEL'] = str_to_date(df_Données_Promos_2021,'DATE_ACTE_REEL') #Preparation join df_Données_Promos_2021 et df_Données_Reabos_2021
    df_join = join_dataFrames(df_Données_Promos_2021,df_Données_Reabos_2021,['ID_ABONNE','DATE_ACTE_REEL']) #Join df_Données_Promos_2021 et df_Données_Reabos_2021

    series_count = mean_time_reabo(df_join,'CPROMO','TIME_REABO') #Creation of a Serie with the mean of the Reabo by CPROMO
    df_filtered_groups = series_count.reset_index() #Changing Series into a  DataFrame 

    df_filtered_groups = df_filtered_groups.sort_values(by = 'TIME_REABO', ascending=False)

    save_to_csv_file(df_filtered_groups,"/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/mean_time_reabo_codes.csv") # Save on My Mac

    return True


In [28]:


def comparaison_time_reabo():


    df_Données_Promos_2021 = file_to_dataframe("/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/Données_Promos_2021.csv")
    df_Données_Reabos_2021 = file_to_dataframe("/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/Données_Reabos_2021.csv")

    end_abo = 'DATE_FIN_ABO_PREC'
    date_reabo = 'DATE_ACTE_REEL'

    df_Données_Reabos_2021['TIME_REABO'] = time_reabo_columns(df_Données_Reabos_2021,end_abo,date_reabo) #Creation new colum "TIME_REABO"

    df_Données_Promos_2021['DATE_ACTE_REEL'] = str_to_date(df_Données_Promos_2021,'DATE_ACTE_REEL') #Preparation join df_Données_Promos_2021 et df_Données_Reabos_2021

    df_join = join_dataFrames_outer(df_Données_Promos_2021,df_Données_Reabos_2021,['ID_ABONNE','DATE_ACTE_REEL']) #Join df_Données_Promos_2021 et df_Données_Reabos_2021 keeping the empty cases

    print(mean_empty_col(df_join,'CPROMO','TIME_REABO')) #Mean time for reabo with and without promotions False mean is promo True mean Na, On peut présenter mieux

    return True
                                        


In [23]:
repartition_reabo_type()

True

In [26]:
repartition_reabo_code()

True

In [29]:
comparaison_time_reabo()

CPROMO
False     9.386528
True     15.479839
Name: TIME_REABO, dtype: float64


True

In [57]:
# 4
#Répartition des réabonnements par taux de consommation (12 mois glissants) par type de promotions et hors promotion

def taux_consommation():
    # Charger le DataFrame à partir du fichier CSV
    df_Données_Reabos_2021 = pd.read_csv("/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/Données_Reabos_2021.csv")

    # Compter le nombre de fois où chaque abonné est abonné
    series_count = df_Données_Reabos_2021.groupby(['ID_ABONNE'])['ID_ABONNE'].count()

    # Créer un DataFrame avec le nombre d'abonnements par abonné
    df_filtered_groups = series_count.reset_index(name='NB_ABOS')

    # Compter le nombre d'abonnés pour chaque nombre d'abonnements
    series_count = df_filtered_groups.groupby('NB_ABOS')['NB_ABOS'].count()
    df_filtered_groups = series_count.reset_index(name='TOTAL_ABO')

    df_filtered_groups = df_filtered_groups.sort_values(by = 'TOTAL_ABO', ascending=False)

    save_to_csv_file(df_filtered_groups, "/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/nb_reabo_2021.csv")

    # 2022

    df_Données_Reabos_2022 = pd.read_csv("/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/Données_Réabos_2022.csv")

    series_count = df_Données_Reabos_2022.groupby(['ID_ABONNE'])['ID_ABONNE'].count()

    df_filtered_groups = series_count.reset_index(name='NB_ABOS')

    series_count = df_filtered_groups.groupby('NB_ABOS')['NB_ABOS'].count()
    df_filtered_groups = series_count.reset_index(name='TOTAL_ABO')

    df_filtered_groups = df_filtered_groups.sort_values(by = 'TOTAL_ABO', ascending=False)

    save_to_csv_file(df_filtered_groups, "/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/nb_reabo_2022.csv")

    # 2023

    df_Données_Reabos_2023 = pd.read_csv("/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/Données_Reabos_2023.csv")

    series_count = df_Données_Reabos_2023.groupby(['ID_ABONNE'])['ID_ABONNE'].count()

    df_filtered_groups = series_count.reset_index(name='NB_ABOS')

    series_count = df_filtered_groups.groupby('NB_ABOS')['NB_ABOS'].count()
    df_filtered_groups = series_count.reset_index(name='TOTAL_ABO')

    df_filtered_groups = df_filtered_groups.sort_values(by = 'TOTAL_ABO', ascending=False)

    save_to_csv_file(df_filtered_groups, "/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/nb_reabo_2023.csv")

    return True
